In [ ]:
!pip install skl2onnx onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.2 which is incompatible.
t

In [ ]:
import onnx

# Check ONNX version
print("ONNX version:", onnx.__version__)

# Create a simple ONNX model and check its validity
dummy_model = onnx.helper.make_model(onnx.helper.make_graph([], "test_model", [], []))
onnx.checker.check_model(dummy_model)

print("ONNX installation is successful and working correctly!")


ONNX version: 1.17.0
ONNX installation is successful and working correctly!


In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import joblib

# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
# Train model
model = LogisticRegression()
model.fit(X_train, y_train)



LogisticRegression()

In [ ]:
# Save model
joblib.dump(model, "logistic_regression_model.pkl")

['logistic_regression_model.pkl']

In [ ]:
import onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Load trained model
model = joblib.load("logistic_regression_model.pkl")

# Define input type for conversion
initial_type = [("float_input", FloatTensorType([None, X_train.shape[1]]))]

# Convert model to ONNX
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save ONNX model
onnx.save(onnx_model, "logistic_regression.onnx")


In [ ]:
import onnxruntime as ort

# Load ONNX model
onnx_session = ort.InferenceSession("logistic_regression.onnx")

# Prepare test input
input_name = onnx_session.get_inputs()[0].name
test_sample = X_test[:5].astype(np.float32)  # Convert to float32 (ONNX requirement)

# Perform inference
predictions = onnx_session.run(None, {input_name: test_sample})
print("ONNX Model Predictions:", predictions[0])


ONNX Model Predictions: [1 0 2 1 1]
